# Mix-up training

paper: https://arxiv.org/abs/1710.09412  
code: https://github.com/facebookresearch/mixup-cifar10

## Environment

In [1]:
%load_ext autoreload
%autoreload 2
%pylab
%matplotlib inline

import pandas as pd
import pickle
import numpy as np
import sys
import os

sys.path.append('/host/projects/sv_experiments/sv_system/')
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="3"

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


### Mix-up algorithm

In [2]:
import torch 

def mixup_data(x, y, alpha=1.0, use_cuda=True):
    '''Returns mixed inputs, pairs of targets, and lambda'''
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size()[0]
    if use_cuda:
        index = torch.randperm(batch_size).cuda()
    else:
        index = torch.randperm(batch_size)

    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam


def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

### Configuration

In [43]:
from utils.parser import set_train_config
import easydict
args = easydict.EasyDict(dict(dataset="gcommand_fbank_xvector",
                              input_frames=70, splice_frames=[20, 70], stride_frames=1, input_format='fbank',
                              cuda=True,
                              lrs=[0.1, 0.01], lr_schedule=[10], seed=1337,
                              no_eer=False,
                              batch_size=128,
                              arch="ResNet34_v1", inplanes=8, loss="softmax",
                              n_epochs=60
                             ))
config = set_train_config(args)

### Dataset and Dataloader

In [44]:
from data.data_utils import find_dataset, find_trial

_, datasets = find_dataset(config, basedir='../../')
trial = find_trial(config, basedir='../../')

In [45]:
from data.dataloader import init_loaders

dataloaders = init_loaders(config, datasets)

### Define Model

In [46]:
from model.model_utils import find_model
model = find_model(config)

In [47]:
model

ResNet34_v1(
  (extractor): Sequential(
    (0): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(3, 3), bias=False)
    (1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (conv2): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (conv2): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), pad

### Train

In [48]:
from train.train_utils import set_seed, find_optimizer
from torch.optim.lr_scheduler import ReduceLROnPlateau

criterion, optimizer = find_optimizer(config, model)
scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.1, patience=5)

In [49]:
set_seed(config)

In [50]:
if not config['no_eer']:
    train_loader, val_loader, test_loader, sv_loader = dataloaders
else:
    train_loader, val_loader, test_loader = dataloaders

In [51]:
from tqdm import tqdm_notebook
from train.train_utils import print_eval

def train(config, train_loader, model, optimizer, criterion):
    model.train()
    loss_sum = 0
    corrects = 0
    total = 0
    print_steps = (np.array([0.25, 0.5, 0.75, 1.0]) \
                    * len(train_loader)).astype(np.int64)

    splice_frames = config['splice_frames']
    if len(splice_frames) > 1:
        splice_frames_ = np.random.randint(splice_frames[0], splice_frames[1])
    else:
        splice_frames_ = splice_frames[-1]

    for batch_idx, (X, y) in tqdm_notebook(enumerate(train_loader), ncols=300,
            total=len(train_loader)):
        # X.shape is (batch, channel, time, bank)
        X = X.narrow(2, 0, splice_frames_)
        X, y_a, y_b, lam = mixup_data(x=X, y=y, alpha=0.4, use_cuda=False)
        if not config["no_cuda"]:
            X = X.cuda()
            y_a = y_a.cuda()
            y_b = y_b.cuda()
        optimizer.zero_grad()
        scores = model(X)
        loss = mixup_criterion(criterion, scores, y_a, y_b, lam)
        loss_sum += loss.item()
        loss.backward()
        # learning rate change
        optimizer.step()
        # schedule over iteration
        predicted = torch.argmax(scores, dim=1)
        corrects += (lam * predicted.eq(y_a).cpu().sum().float()
                    + (1 - lam) * predicted.eq(y_b).cpu().sum().float())
        total += y_a.size(0)
        if batch_idx in print_steps:
            print("train loss, acc: {:.4f}, {:.5f} ".format(corrects/total, loss_sum))
            
    return loss_sum, corrects/total

In [53]:
from train.si_train import val, sv_test

for epoch_idx in range(0, config['n_epochs']):
    print("-"*30)
    curr_lr = optimizer.state_dict()['param_groups'][0]['lr']
    
#     #lr_scheduling
#     idx = 0
#     while(epoch_idx >= config['lr_schedule'][idx]):
#     # use new lr from schedule epoch not a next epoch
#         idx += 1
#         if idx == len(config['lr_schedule']):
#             break
#     curr_lr = config['lrs'][idx]
#     optimizer.state_dict()['param_groups'][0]['lr'] = curr_lr
    print("curr_lr: {}".format(curr_lr))

#     train code
    train_loss, train_acc = train(config, train_loader, model, optimizer, criterion)

#     validation code
    val_loss, val_acc = val(config, val_loader, model, criterion)
    
    print("epoch #{}, train accuracy: {}".format(epoch_idx, train_acc))
    print("epoch #{}, val accuracy: {}".format(epoch_idx, val_acc))

#     evaluate best_metric
    if not config['no_eer']:
        # eer validation code
        eer, label, score = sv_test(config, sv_loader, model, trial)
        print("epoch #{}, sv eer: {}".format(epoch_idx, eer))
    
    scheduler.step(train_loss)

------------------------------
curr_lr: 0.1



train loss, acc: 0.2258, 309.71641 
train loss, acc: 0.2249, 615.93158 
train loss, acc: 0.2238, 918.52456 

epoch #0, train accuracy: 0.228590726852417
epoch #0, val accuracy: 0.18975013494491577
epoch #0, sv eer: 0.15456666666666666
------------------------------
curr_lr: 0.1


train loss, acc: 0.2201, 317.64663 
train loss, acc: 0.2350, 611.59307 
train loss, acc: 0.2432, 902.47225 

epoch #1, train accuracy: 0.25156718492507935
epoch #1, val accuracy: 0.28307169675827026
epoch #1, sv eer: 0.1453888888888889
------------------------------
curr_lr: 0.1


train loss, acc: 0.2071, 326.33787 
train loss, acc: 0.2092, 638.52274 
train loss, acc: 0.2139, 946.46682 

epoch #2, train accuracy: 0.21531441807746887
epoch #2, val accuracy: 0.23835326731204987
epoch #2, sv eer: 0.16526666666666667
------------------------------
curr_lr: 0.1


train loss, acc: 0.2117, 320.21239 
train loss, acc: 0.2031, 642.07108 
train loss, acc: 0.2054, 958.71809 

epoch #3, train accuracy: 0.21264539659023285
epoch #3, val accuracy: 0.1779341846704483
epoch #3, sv eer: 0.17201111111111111
------------------------------
curr_lr: 0.1


...

train loss, acc: 0.5564, 212.08978 
train loss, acc: 0.5523, 426.62940 
train loss, acc: 0.5675, 605.25932 

epoch #55, train accuracy: 0.5746539235115051
epoch #55, val accuracy: 0.7229370474815369
epoch #55, sv eer: 0.11591111111111112
------------------------------
curr_lr: 1.0000000000000004e-05


train loss, acc: 0.5668, 202.69427 
train loss, acc: 0.5812, 397.92837 
train loss, acc: 0.5890, 578.59296 

epoch #56, train accuracy: 0.5872601866722107
epoch #56, val accuracy: 0.7414674758911133
epoch #56, sv eer: 0.11478888888888888
------------------------------
curr_lr: 1.0000000000000004e-05


train loss, acc: 0.5601, 203.53050 
train loss, acc: 0.5578, 403.63675 
train loss, acc: 0.5564, 611.96360 

epoch #57, train accuracy: 0.5671892762184143
epoch #57, val accuracy: 0.7294138669967651
epoch #57, sv eer: 0.11481111111111111
------------------------------
curr_lr: 1.0000000000000004e-06


train loss, acc: 0.5683, 204.35761 
train loss, acc: 0.5613, 408.58376 
train loss, acc: 0.5545, 623.88112 

epoch #58, train accuracy: 0.5425592064857483
epoch #58, val accuracy: 0.654853105545044
epoch #58, sv eer: 0.11656666666666667
------------------------------
curr_lr: 1.0000000000000004e-06


train loss, acc: 0.6320, 189.06479 
train loss, acc: 0.6513, 357.25658 
train loss, acc: 0.6804, 498.36908 

epoch #59, train accuracy: 0.6934493184089661
epoch #59, val accuracy: 0.7463781833648682
epoch #59, sv eer: 0.11308888888888889


In [19]:
torch.save(model.cpu().state_dict(), open("model_repos/mixup_train_gcommand_0.8.pt", "wb"))

In [21]:
ddmodel.cuda()

ResNet34_v1(
  (extractor): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(3, 3), bias=False)
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace)
        (conv2): Conv2d(16, 16, kernel_size=(3, 3), strid